# Correlation and Regression

In [ ]:
from IPython.display import Markdown
base_path = (
    "https://raw.githubusercontent.com/rezahabibi96/GitBook/refs/heads/main/"
    "books/applied-statistics-with-python/.resources"
)

In [ ]:
import math
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.utils import resample
from scipy.stats import norm, binom, chi2, chisquare, chi2_contingency, expon, shapiro, t, wilcoxon
from statsmodels.stats.power import TTestPower, TTestIndPower

import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.pyplot import figure

import requests
from io import BytesIO

In this chapter, we introduce methods for determining whether there is a linear correlation, or association, between two numerical variables. In the case it exists, we learn how to derive a linear model and how to use it for prediction, which is very important in applications of many different fields from Business and Finance to Biology and Medicine.
For example:
* Based on income, consumer debt, and several other predictors about a person, we might be interested in predicting a person's credit rating.
* We might be interested in predicting growth in sales of a company based on current economic conditions and its previous year's performance.
* In Medicine, it would be important to predict patients' blood pressure based on health measurements and the dosage of a new drug.

In fairness, the more interesting examples above have several predictors, which is called multiple regression and is taken up in detail in later chapters. In this chapter, we study correlation in general, but the linear models are limited to a single predictor.

## Correlation

As an example, let's return to the file `MHEALTH.csv` mentioned before in chapters 1 and 2. It contains many health measurements for 40 men. Let's investigate weight (WT) dependence on the waist size (WAIST).

Whenever you plan a correlation/regression study, you *must always start with a scatterplot* as shown in the figure below.

There is a fair amount of variation, but there seems to be an overall linear trend with a positive slope. Not surprisingly, a man with a larger waist size has a higher weight.

The numerical measure of the strength of this *linear* trend is given by the **Pearson correlation coefficient**:

$$ r = \frac{1}{n-1} \sum_{i=1}^n (z_x)_i \cdot (z_y)i = \frac{1}{n-1} \sum{i=1}^n \frac{x_i - \bar{x}}{s_x} \cdot \frac{y_i - \bar{y}}{s_y} $$

where $n$ is the number of data point pairs. The `scipy.stats` library has an effective function to perform this rather tedious computation.

Note that because the standardized z-scores are used to define $r$, it is a dimensionless number that does not depend on units of measurement of either variable.
For example, assume that $x$ is the square footage ($ft^2$) of an apartment, and $y$ is its price in dollars. If we change $x$ to square meters and/or $y$ to thousands of dollars, the correlation coefficients would not change.
Also, $x$ and $y$ come symmetrically into the $r$ formula, so they can be interchanged.

The correlation coefficient is always between $-1 \le r \le 1$, and the figure below illustrates different sample scatterplots and their correlations.

A set of heuristic bounds for the correlation coefficient could be defined as follows:
1. $0 \le |r| \le 0.3$ — no correlation to weak correlation
2. $0.3 \le |r| \le 0.6$ — weak correlation to moderate correlation
3. $0.6 \le |r| \le 1$ — moderate correlation to strong correlation

These bounds are imprecise and very much depend on the field of study. In a science or an engineering experiment, when the variables are related by a Physics law, a correlation must be very high, close to $-1$ or $1$. On the other hand, in business, economics, medicine, etc., with all the variability in human behavior, even a correlation of $0.6$ could be considered strong.
In the figure above, top to bottom, left to right, the correlation coefficient decreases from perfect positive to strong, moderate, weak, no correlation, and similarly for negative correlations.
Note that Pearson correlation coefficient $r$ only quantifies **linear** correlation. It **can't** describe any **non-linear** relationship. The $r$ could still be computed, but it does not make much sense as shown in the figure below.

The formula for the correlation coefficient is
$ r = \frac{1}{n-1} \sum_{i=1}^n (z_x)_i \cdot (z_y)_i $
and the figure below can help understand the sign and magnitude of the correlation.

For the positive slope (uphill) line, $z_x$ and $z_y$ are either both $> 0$ or both $< 0$; therefore the $\sum z_x \cdot z_y$ is positive and large. Analogously, for the negative slope (downhill) line, $z_x$ and $z_y$ have opposite signs, so $\sum z_x \cdot z_y$ is negative and large. For the points in the middle plot, which are all over the place, $z_x \cdot z_y$ have different signs and cancel each other out, so $\sum z_x \cdot z_y$ is close to 0. This sum is divided by $n-1$ to average, which leads to bounds $-1 \le r \le 1$.

The sample correlation coefficient $r$ is an approximation of the true population correlation coefficient $\rho$. The hypothesis test for $\rho$ is:

$H_0$: $\rho = 0$ — no linear correlation

$H_1$: $\rho \ne 0$ — there is a linear correlation

The hypothesis is tested with t-distribution with $df = n-2$ degrees of freedom:

$$ SE = \sqrt{\frac{1-r^2}{n-2}} $$

$$ t = \frac{(\text{point estimate}) - (\text{null value})}{SE} = \frac{r - 0}{SE} = \frac{r}{SE} $$

Note that the Pearson's $r$ hypothesis test is a parametric test, and the data must satisfy the usual assumptions that both variables follow normal distributions. At least no outliers should dramatically change $r$. Such outliers are influential observations, which will be discussed later in this chapter.

For example, for WT and WAIST size in the MHEALTH data file, the above formulas give:

Therefore, WAIST and WT are strongly positively linearly correlated. Also, $p$-value $< 0.05$, so we can reject $H_0: \rho = 0$ (no linear correlation) to claim that this correlation is significant.

On the other hand, for WT and PULSE, there is practically no correlation and $p$-value $> 0.05$ (non-significant):

Note that the strength of the correlation is given by the magnitude of $r$, while the significance is determined by the $p$-value of the corresponding t-test. They are related, but not the same. Say, for a large data set, even a weak correlation coefficient can be statistically significant.

Many possible pairwise correlation coefficients exist in the case of several interrelated numerical variables. The code below gives an efficient way to obtain all the pairwise correlations at once and have their visual representation.

Therefore, WT is weakly correlated with AGE, not correlated with CHOL (cholesterol), moderately correlated with HT (height), and strongly positively correlated with BMI. We can check individual significance with $p$-value, for example for WT and HT:

Moreover, the square of $r$ — $r^2$ — is called the **coefficient of determination** and has an important meaning. It is the proportion of the variation in the response variable $y$ that is explained by the linear relationship between $x$ and $y$.

For example, for the relationship between HT and WT, $r^2 = 0.5222^2 = 0.273$. Therefore, 27.3% of variation in WT can be explained by the linear relationship with HT.

On the other hand, for the relationship between WT and PULSE, $r^2 = 0.0562^2 = 0.0031$. Therefore, only 0.31% of variation in WT can be explained with the linear relationship with PULSE (no relationship).

To repeat the point that was made in the beginning, **correlation does NOT imply causation**. Only a randomized experiment can be used to assess the causality!

### Spearman Correlation

In the case when the samples are small and/or skewed, the normality cannot be assumed, and non-parametric tests that make no such assumptions are in order. Spearman rank-order correlation can evaluate the monotonic relationship between two continuous or ordinal variables. In such a relationship, the variables tend to change together, but not necessarily at a constant rate.
Much like other non-parametric tests (Wilcoxon, Mann-Whitney, etc.), the Spearman correlation coefficient is based on the ranked values rather than the raw data. As a ranking, it can also be naturally used for ordinal variables like the Likert Scale: 
(1 — Strongly Disagree,
2 — Disagree,
3 — Neutral,
4 — Agree,
5 — Strongly Agree)

As an example, consider a data file on a possible correlation between the number of medications a person took in the past week (discrete counting variable, not normally distributed) and the score on a standard memory recall test. We expect that the greater the number of medications, the worse the patients' memory will be; therefore a one-tailed test is in order. The data set is small.

The correlation coefficient is negative (as predicted), but it is weak and not statistically significant at any acceptable level of significance. Note that we have to divide the $p$-value which we obtained by 2 (one-sided test), but it is still way more than 0.05. Thus, the number of medications was not significantly related to memory performance.

Kendall's rank correlation tests the similarities in the ordering of data ranked by quantities. Pearson and Spearman use the individual observations as the basis of the correlation, while Kendall's correlation coefficient uses pairs of observations. It determines the strength of association based on the pattern of concordance and discordance between the pairs. For the medication problem above, the conclusions are the same.

### Partial Correlation

Partial correlation is the correlation between two variables while controlling for a third (or more) variable.
Consider an example data set of `Income` and `Performance` scores at a job together with `Experience` in years. As we can see in the correlation matrix figure below, the correlations computed without partialling anything out are very strong between all three variables.

In the above output, the years of `Experience` have been covaried (partialled out). The correlation between Income and Performance score has been reduced considerably to less than half of the original one, and the $p$-value $> 0.05$, so it is not significant anymore.
It removed that part of the correlation between `Income` and `Performance` score which was due to years of `Experience`, and there is not much correlation left.

The correlation above has been reduced to a moderate 0.61, but it is still significant

The correlation above is not significant anymore.

We can illustrate this idea using the diagram above showing overlapping circles for each of our three variables. It is not mathematically accurate; the circles are exaggerated to understand the idea.
The shared variance between `Income` and `Performance` is represented by the areas a and b.
The `Experience` is related (shares variance) with both `Income` and `Performance` in areas c, b, and d, respectively.
Area a shows the unique variance of `Income` and `Performance`.
Area b represents the part of the relationship between `Income` and `Performance` which is due to Experience.
If we remove the influence of Experience (area b), the correlation between `Income` and `Performance` is considerably reduced (area a only).

A medical example of partial correlation might be a relationship between symptoms and quality of life, with depression partialled out (covaried). If the correlation between symptoms and quality of life is considerably reduced after partialling out depression, it can be concluded that a part of the relationship between symptoms and quality of life is due to depression.

## Least Squares Regression Line

Given that there is a strong correlation between `WT` and `WAIST`, the observations follow a linear model. What is the equation of the best line? We denote it as:

$$ \hat{y} = E(Y \mid X = x) = b_0 + b_1 x $$

To find the best line, a **residual** for each data point $i = 1\dots n$ is defined as the vertical distance between the actual $y$-value $y_i$ of the data point and its prediction by the line $\hat{y}_i$:

$$ e_i = y_i - \hat{y}_i = y_i - (b_0 + b_1 \cdot x_i) $$

The figure below shows data with the best regression line on the left side and residuals on the right side using 'sns.regplot()' and `sns.residplot()`, respectively.

The residuals seem to be randomly positive and negative around $y = 0$, so the line fits the data well. The positive and negative residuals would cancel each other in a sum. Instead, a sum of squared residuals: $e_1^2 + e_2^2 + \dots + e_n^2$ is minimized (**least squares regression line**). The idea is analogous to defining the standard deviation using the sum of squared deviations from the mean:
$ s = \sqrt{\frac{\sum (x_i - \bar{x})^2}{n-1}} $
The minimization is done using Multivariable Calculus. The slope is:

$$ b_1 = \frac{s_y \cdot r}{s_x} $$

where $r$ is the correlation coefficient, and $s_x$ and $s_y$ are standard deviations.
The proportionality and positivity of standard deviations imply that the sign of the slope of the line is the same as the correlation coefficient $r$ sign. Also, it is shown that the means point $(\bar{x}, \bar{y})$ is always on the regression line. Therefore, the point-slope form of the equation of regression line is:

$$ y - \bar{y} = b_1 (x - \bar{x}) \implies y = \bar{y} - b_1 \bar{x} + b_1 x $$

Therefore, the y-intercept is:

$$ b_0 = \bar{y} - b_1 \bar{x} $$

Let's illustrate these formulas:

Based on the code above, the y-intercept is $b_0 = -44.08$, and the slope is $m = b_1 = 2.37$. Therefore:

$$ \hat{y} = b_0 + b_1 x = -44.08 + 2.37 \cdot x $$

or in terms of the original variables:

$$ \hat{WT} = -44.08 + 2.37 \cdot WAIST $$

A slope is a rise over run; therefore here, for each 1 cm increase in `WAIST`, you are expected to gain **on average** 2.37 lb of `WT`. Note that for the linear model, it matters that **explanatory (independent) variabl*e* `WAIST` explains **response (dependent) variable** `WT`, while for correlation it did not matter.

Let's find the residual for a given man with `WAIST` size of 95 cm and weight of 181 lb (data point $(95, 181)$):

$$ \hat{y} = -44.08 + 2.37 \cdot 95 \approx 181.37 $$

Therefore, the residual is: 

$$ e = y - \hat{y} = 181 - 181.37 = -0.37 < 0 $$

Now, for a different man with `WAIST` size of 75 cm and weight of 170, the residual is positive:

$$ \hat{y} = -44.08 + 2.37 \cdot 75 \approx 133.90 $$
$$ e = y - \hat{y} = 170 - 133.90 = 36.1 > 0 $$

Generally, if a prediction is below the actual data point (*underestimate*), then $e = y - \hat{y} > 0$ — positive residual. On the other hand, if a prediction is above the actual data point (*overestimate*), then $e = y - \hat{y} < 0$ — negative residual.

### Assumptions for the Least Squares Model

1. Linearity. The scatterplot must have a linear trend, and the corresponding residual plot should be random (no regular pattern). The simulation below shows: A linear trend in the data with a corresponding random residual plot without any pattern (left side of the figure below). A non-linear pattern in the data and residuals (right side of the figure below).

2. **Normal Residuals**. The residuals must be approximately normally distributed. Usually, it is very influential outliers that fail this condition and may completely invalidate the linear model. We discuss the classification of leverage and the influence of such outliers in more detail shortly. The figure below shows the data with dramatic outliers which destroy the correct linear trend in the regression line and show up as extreme right-skew outliers in the residual histogram below.

3. **Constant Variability**. The variability of observations about the least squares line should remain approximately the same. The figure below shows the data where the variability increases (**funnel pattern**).

4. **Independent Observations**. The observations must be independent - one does not affect the other. The least squares line model cannot explain the time series data (sequential observations in time such as gasoline prices over several years or stock price variations in time). There are specialized time series analysis techniques for such data. The figure below shows the data with oscillating time dependence (seasonal changes on top of linear trend). In addition, the Durbin-Watson test can be used to check for autocorrelation. It reports a test statistic with a range from 0 to 4, where the value of 2 indicates that there is no autocorrelation. The (0,2) range corresponds to positive autocorrelation (common in time series data), and the (2,4) range indicates negative autocorrelation (less common in time series data).

###  Statistical Analysis of Regression Coffcients

Assuming the data set satisfies all the assumptions of the linear regression above, we can analyze the goodness of fit of the linear model in several ways: correlation, slope, ANOVA, etc. The correlation analysis was already done in the previous section.

The slope hypothesis test:

$H_0: \beta_1 = 0$ - zero slope, no linear relationship

$H_1: \beta_1 \ne 0$ - non-zero slope, there is a linear relationship

The standard error of the slope is:

$$SE(b_1) = \sqrt{\frac{1}{n-2} \frac{\sum_{i=1}^{n} (y_i - \hat{y}i)^2}{\sum_{i=1}^{n} (x_i - \bar{x})^2}}$$

The regression summaries for `WT` vs. `WAIST` are given in the extensive tables below and the figure shows the scatterplot with the best fit line.

The diagnostic plots above provide extremely important data summaries. The top-left one is Residuals vs. Fitted values. This plot checks if residuals have non-linear patterns. If you find equally spread residuals around a horizontal line without distinct patterns (as in this case), that is a good indication we don't have a non-linear relationship.
The top-right plot shows a quantile-quantile (QQ) plot of standardized residuals vs. the standard normal distribution quantiles. If the residuals approximately follow a straight line (as in this case) the normality assumption of residuals holds.
The bottom-left figure is the Scale-Location plot. It checks if residuals are spread equally along the ranges of predictors - assumption of equal variance (homoscedasticity). The top-left figure shows this too, but this one shows the possible funnel pattern more clearly. This example shows the line with equally (randomly) spread points, so the homoscedasticity holds.
The bottom-right figure shows Residuals vs. Leverage. We discuss it in more detail in the next section.

The "Coef" part of the table gives the y-intercept, slope, and the corresponding statistical analysis. For the slope:

$$t = \frac{\text{point estimate} - \text{null value}}{SE} = \frac{b_1 - 0}{SE(b_1)} \approx \frac{2.37}{0.2} \approx 11.96$$

The t-distribution for the slope with $df = n - 2 = 40 - 2 = 38$ degrees of freedom produced an extremely low **p-value** $= 1.87 \cdot 10^{-14} < 0.05$, so the null hypothesis of zero slope $\beta_1 = 0$ can be rejected (significant linear model). Note also that the t-test and p-value are exactly the same as for the correlation hypothesis. It is not an accident because for simple one-variable regression $b_1 = \frac{s_y r}{s_x}$, i.e., slope is proportional to the correlation coefficient.
In addition, slope standard error can be used to define the confidence interval:

$$\text{point estimate} \pm t_{df} \cdot SE = b_1 \pm t_{n-2} \cdot SE(b_1)$$

$$= 2.37 \pm 2.024 \cdot 0.2 = (1.97, 2.77)$$

This confidence interval does not contain 0 which proves, yet again, that the slope is not 0 and the linear model is valid.

The code output also shows an analogous statistical analysis for the y-intercept $H_0: \beta_0 = 0$, which is rarely used (unless we look for direct proportionality). It is the output of the linear model when input is zero, but $x = 0$ is often impractical. For example, in our `WT` vs. `WAIST` example, `WAIST` = 0 is not practical and the resulting $b_0 = -44.08$ corresponds to negative weight which does not make sense either. We will see some other examples in future sections where $b_0$ has a practical meaning.

### Leverage and Influence

Generally, outliers in regression are observations that fall far from the regression line. However, the leverage and influence of such outliers must be investigated.
1. **Leverage**. Points that fall horizontally far away from the center (mean) of the data cloud pull harder on the line - high-leverage points, so leverage measures the extremity of the predictors. A high-leverage point is an observation with x-values extreme enough to *potentially* significantly change the slopes or trends in the fitted model. An outlier can have a high or low leverage.

The formula for leverage is:

$$h_i = \frac{1}{n} + \frac{(x_i - \bar{x})^2}{s_x^2}$$

where $n$ is the sample size of $(x, y)$ pairs, $x_i$ is the x-coordinate of a particular point, $\bar{x}$ is the mean of all x's, and $s_x^2$ is the variance of x's.

In general $1/n \le h_i \le 1$. If there are $p$ independent variables in the model, the mean value for leverage is $(p+1)/n$ (for simple regression $p=1$, so it is $2/n$). A rule of thumb (Steven's) is that values that are 3 times this mean value are considered large.

2. **Influence**. An observation with high leverage that significantly affects the regression line is influential. The standard measure of influence is Cook's distance, which estimates the magnitude of the effect of deleting the $i$-th observation from the model. Cook's distance is defined in a more general context of multiple regression with $p$ explanatory variables (predictors), in our case $p = 1$.

Let:

$\hat{y}_j$ = predicted mean response of observation $j$ for the model fitted with all $n$ observations.

$\hat{y}_j^{(-i)}$ = same response fitted without the $i$-th observation.

$se$ = residual standard error. 
Then Cook's distance is:

$$D_i = \frac{\sum_{j=1}^{n} (\hat{y}_j - \hat{y}_j^{(-i)})^2}{(p+1) s_e^2}$$

The Residuals vs. Leverage diagnostic plot with Cook's distances shown as dashed level curves with levels 0.5, 1, etc. illustrates the interplay between leverage and influence. The figure below shows the effect of adding one point (big square) to the original data on the scatterplot and Residuals vs. Leverage plots. Highly influential points cross Cook's level curves as shown in the bottom-right plot below.

In the top plots of the above figure, the additional point has low leverage since its predictor value is close to $\bar{x}$. Its inclusion only slightly modifies the y-intercept of the original line, not the slope, so it has very low influence, as can be seen in the top-right graph with none of the points crossing Cook's distance level 0.5.
In the middle plots, the additional point is far apart from the original observations' mean, so it has high leverage. However, it conforms very well to the regression line model fitted without it. Its inclusion changes the regression line very slightly, so the influence is still low and it does not cross Cook's distance level 0.5.
Lastly, in the bottom plots, the additional point is very far away from the original observations and has high leverage. This time, however, its inclusion significantly alters the regression line by turning it clockwise, so it has very high influence and it crosses Cook's distance level curves 0.5 and 1.

### Regression ANOVA

Next, we introduce an Analysis of Variance (ANOVA) for regression. It is studied in much more detail in the context of comparing several means, but it can illuminate the goodness of fit of the regression line as well. Assume there is a set of observations (not shown in the figure below) that produces the regression line $\hat{y} = 9 + 1.5x$. Concentrate on just one data point $(2, 15)$ to keep the figure clear. Also, the overall mean of the response variable is $\bar{y} = 10$.

The total vertical deviation of data point $(2, 15)$ from overall response mean is $y_i - \bar{y} = 15 - 10 = 5$. It can be broken into the sum of two deviations:

$$\text{total deviation} = \text{explained deviation} + \text{unexplained deviation (residual)}$$

$$y_i - \bar{y} = (\hat{y}_i - \bar{y}) + (y_i - \hat{y}_i)$$

$$5 = 2 + 3$$

As always, deviations are both positive and negative and would cancel each other out in a sum, so squared deviations are considered. The total sum of squares can be broken into:

$$SST (\text{total}) = SSR (\text{explained or regression}) + SSE (\text{residual or error})$$

$$\sum_{i=1}^n (y_i - \bar{y})^2 = \sum_{i=1}^n (\hat{y}i - \bar{y})^2 + \sum{i=1}^n (y_i - \hat{y}_i)^2$$

The ANOVA results are best represented in a table below that shows degrees of freedom, sums of squares, averaged squared deviations (sum of squares divided by the corresponding degrees of freedom), F ratio of averaged deviations, and the corresponding p-value.

For **simple regression** with only one explanatory variable, the F-test ($F = t^2$, i.e., $143.12 \approx 11.96^2$) and the p-value is the same as for the slope hypothesis t-test. Thus, this confirms, yet again, the validity of the linear model. This is not true for multiple regression where the F-test provides overall significance and each variable has its own t-test.
Moreover, the averaged residual sum of squares
$MME = \frac{1}{df} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$
in the ANOVA can be used to calculate the residual standard error:

$$\text{Residual standard error} = \sqrt{MME} = \sqrt{\frac{1}{df} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

For `WT` vs. `WAIST`, the residual standard error is $\sqrt{149.30} = 12.22$.

When correlation was introduced, the **coefficient of determination** $r^2$ was defined, which measures the proportion of the variation in the response variable that is explained by the linear model. More precisely:

$$r^2 = \frac{SSR}{SST} = \frac{21360.1}{21360.1 + 5671} = 0.79019$$

which is the ratio of the explained variation in ANOVA to the total variation. It is exactly the $r^2$ that we obtained before.

### Linear Model Prediction

If a linear model is appropriate for the data, it can be used to predict the response for a given explanatory variable. However, it is only a point estimate, while a confidence interval is needed. There are actually two different kinds of confidence intervals for the prediction.

**Confidence interval for the individual response value.**

First, suppose we consider a particular man (John) with a `WAIST` size of 81 cm. For our `WT` vs. `WAIST` model, the `WT` predicted by the least squares line is simply found by plugging 81 into the regression line equation:

$$ y^* = b_0 + b_1 \cdot x^* = -44.08 + 2.37 \cdot 81 = 148.14 $$

The regression model is based on a particular sample of $n=40$ men and would be a bit different for a different sample. The sample variation can be quantified via a confidence interval.

$$(point estimate) ± t_{df} \cdot SE = y^* \pm t_{df} \cdot SE$$

The standard error formula is given by

$$ SE = \sqrt{s_e^2 + \frac{s_e^2}{n} + SE(b_1)^2 (x^* - \bar{x})^2} $$

where $s_e$ is residual standard error and $SE(b_1)$ - slope standard error. The $s_e^2$ term accounts for variation among all of the residuals, while $\frac{s_e^2}{n}$ accounts for variation in the particular residual for $y^*$. Any linear regression model is anchored at the means point $(\bar{x}, \bar{y})$. As the given $x^*$ gets further away from the mean $\bar{x}$, the term $SE(b_1)^2 (x^* - \bar{x})^2$ becomes larger, and the standard error increases producing a wider confidence interval.
For our example:

**Confidence interval for the mean response value.**

The mean values are more predictable than individual measurements (based on the Central Limit Theorem), so in predicting the mean *WT* of men with *WAIST* 81 cm, we can expect a smaller standard error SE:

$$ SE = \sqrt{\frac{s_e^2}{n} + SE(b_1)^2 (x^* - \bar{x})^2} $$

It is almost the same formula as for the individual prediction, except it does not have the term $s_e^2$, which accounts for variation among all of the residuals.

Both, the individual prediction confidence interval and the mean prediction confidence interval are really computed by a single command:

In fact, we can compute prediction intervals for several values at once as shown in the code below:

The figure below shows 95% confidence intervals for the mean prediction around the regression line and the individual prediction (much wider). As the point moves further away from the mean $\bar{x}$, the confidence intervals get wider and wider.

## Linear Model Examples

**Example**

Investigate correlation and regression for the data below on Longevity and Smoking. Predict several typical values.

Based on the diagnostic plots and the data with regression line figure above, we conclude the following.
The assumptions of regression:
a linear trend (no pattern of residuals),
normality of residuals (QQ plot),
constant variability (no funnel pattern),
independent observations (no time series, Durbin-Watson is close to 2),
are all satisfied. Residuals vs. Leverage does not have any points over Cook's distance level curves. Thus, the linear model is applicable.

The correlation coefficient $r = -0.888$ indicates a strong negative correlation.

Correlation hypothesis test:

$H_0: \rho = 0$ no linear correlation

$H_1: \rho \neq 0$ there is linear correlation

The t-test gives $t = -7.24$ and corresponding p-value $= 4.3230332 \cdot 10^{-6} < 0.05$, therefore $H_0$ is rejected, and there is a significant linear correlation.
$ r^2 = 0.789 $; therefore 78.9% of variation in longevity is explained by this linear model (coefficient of determination).

Slope $b_1 = -4.04$; therefore for each extra pack smoked per day, on average, the longevity decreases by 4.04 years.
The hypothesis test for the slope:

$H_0: \beta_1 = 0$ zero slope, no linear relationship

$H_1: \beta_1 \neq 0$ non-zero slope, there is a linear relationship

The same t-test $t = -7.24$ and p-value $= 4.3230332 \cdot 10^{-6} < 0.05$ lead to rejection of $H_0$; so there is a significant linear relationship.

The 95% confidence interval for the slope $(-5.24, -2.84)$ doesn't contain 0, which is yet another indication of the non-zero slope.

The y-intercept $b_0 = 75.39$ has practical meaning, unlike most other problems. It is the expected value of longevity for a non-smoker (packs smoked per day equal to 0).

Thus, it has been shown in several ways that the linear model $\hat{y} = 75.39 - 4.04 \cdot x$ or $\hat{Longevity} = 75.39 - 4.04 \cdot Packs$ is appropriate for these data, so it can be used for prediction.

The code below gives the predictions fit as well as individual prediction confidence intervals and tighter mean confidence intervals.

We can also compute a residual for a specific value slightly outside of the data set (reasonable extrapolation) as well as the typical residual error (Residual standard error $s_e$).

**Example**

Consider correlation and regression for the `WT` vs. `CHOL` level in the `MHEALTH` file. If the linear model is good, predict several typical values.

Most importantly, there is *NO* linear trend as shown in the data with regression line figure above. Also, based on the diagnostic plots above, the normality of residuals (QQ plot) is approximately satisfied. The constant variability seems to fail; there is a funnel pattern. Independent observations (no time series, although Durbin-Watson is a bit below 2 indicating some autocorrelation) seem to be satisfied. The residuals vs. Leverage plot also shows no extreme behavior. With no linear trend, there is no reason to continue; however, we pursue to see how each test fails in detail.

Correlation coefficient $r = -0.026$ implies almost no correlation.
Correlation hypothesis test:

$H_0: \rho = 0$ no linear correlation

$H_1: \rho \neq 0$ there is a linear correlation

The t-test gives $t = -0.16$ and corresponding *p-value* $= 0.8739101 > 0.05$, so $H_0$ cannot be rejected, and there is no significant linear correlation.

The coefficient of determination $r^2 = 0.00067$; therefore only 0.067% of variation in `WT` is explained by this linear relationship with `CHOL` (coefficient of determination).

The hypothesis test for the slope:

$H_0: \beta_1 = 0$ zero slope, no linear relationship

$H_1: \beta_1 \neq 0$ non-zero slope, there is a linear relationship

The same t-test $t = -0.16$ and corresponding *p-value* $= 0.8739101 > 0.05$ tell us again not to reject $H_0$; so there is no significant linear relationship.

In addition, the ANOVA test gives $F = 0.026$ with the same p-value, confirming our conclusion yet again. The 95% confidence interval for the slope is $(-0.03, 0.03)$, which contains 0 — yet another indication of a non-significant slope.

Because this linear model is not appropriate, it should *NOT be used for prediction*.

**Example**

Investigate correlation and regression for the `HELPrct` substance abusers data on depression score `cesd` vs. mental health score `mcs`. Predict several typical values.

Based on the diagnostic plots and the data with regression line figure above, the following conclusions can be made.
The assumptions of regression:
linear trend (no pattern of residuals),
normality of residuals (QQ plot),
constant variability (no funnel pattern),
independent observations (no time series),
are all satisfied. In the Residuals vs. Leverage plot, none of the points go over Cook's distance level curves (they are not even visible); so none of the observations are too influential. Thus, the linear model is applicable.

Pearson correlation coefficient $r = -0.682$ corresponds to a moderate negative correlation.

Correlation hypothesis test:

$H_0: \rho = 0$ — no linear correlation

$H_1: \rho \neq 0$ — there is linear correlation

The t-test produces $t = -19.8$ and the corresponding *p-value* $\approx 0 < 0.05$; therefore, $H_0$ can be rejected, i.e., there is a significant linear correlation.

The coefficient of variation is $r^2 = 0.465$; therefore, 46.5% of variation in `cesd` is explained by this linear relationship with mcs (coefficient of determination).

Slope $b_1 = -0.665$, so for each extra point of `mcs` score, the depression score `cesd` decreases by 0.665 *on average*.
The hypothesis test for the slope:

$H_0: \beta_1 = 0$ — zero slope, no linear relationship

$H_1: \beta_1 \neq 0$ — non-zero slope, there is a linear relationship

The same t-test $t = -19.8$ and *p-value* $\approx 0 < 0.05$ lead to the rejection of $H_0$, i.e., there is a significant linear relationship.

The 95% confidence interval for the slope $(-0.73, -0.6)$ doesn't contain 0, which is yet another indication of the non-zero slope.

Having confirmed in several ways that the linear model $\hat{y} = 53.90 - 0.665 \cdot x$ is appropriate, let's predict with it:

The output above shows both tighter mean predictions and wider individual predictions.

We can also compute a residual for a specific value slightly outside of the data set (reasonable extrapolation). The typical residual error using this linear model is given by Residual standard error $s_e$ computed below.

**Example**

Investigate the correlation and regression of a particular stock return vs. the S&P500 stock average. Predict several typical values.

Based on the diagnostic plots and the data with regression line figure above, the following conclusions can be made.
The assumptions of regression:
linear trend in the data (no pattern of residuals),
normality of residuals (QQ plot),
constant variability (no funnel pattern),
independent observations (no time series),
are satisfied. In the Residuals vs. Leverage plot, none of the observations cross over Cook's distance level curves. Thus, we can apply the linear model to this data.

Pearson correlation coefficient $r = 0.954$ indicates a strong positive correlation.

Correlation hypothesis test:

$H_0: \rho = 0$ — no linear correlation

$H_1: \rho \neq 0$ — there is a linear correlation

The t-test produces $t = 12.66$ and corresponding *p-value* $\approx 0 < 0.05$; therefore, $H_0$ can be rejected and there is a significant linear correlation.

The coefficient of determination is $r^2 = 0.909$, so 90.9% of the variation in returns of the stock ABC is explained by this linear relationship with returns of S&P500.

Slope $b_1 = 0.59$, so for each extra point of return of S&P500, the returns of stock ABC increase by 0.59 on average.

The hypothesis test for the slope:

$H_0: \beta_1 = 0$ — zero slope, no linear relationship

$H_1: \beta_1 \neq 0$ — non-zero slope, there is a linear relationship

The t-test is the same $t = 12.66$ and *p-value* $\approx 0 < 0.05$, so $H_0$ can be rejected, and there is a significant linear relationship.

The 95% confidence interval for the slope $(0.49, 0.69)$ doesn't contain 0, which is yet another indication of the non-zero slope.

Having confirmed in several ways that the linear model $\hat{y} = -0.0059 + 0.5894 \cdot x$ fits the data well, use it for prediction.

In the code below, we investigate a residual for a particular value. Also, the typical residual error is given by Residual standard error $s_e$.

For the standardized stock returns in this problem, the slope of the linear model $b_1 = 0.59$ gives the **beta of a stock**, which measures the volatility of the stock relative to the stock market as a whole (represented via S&P500). Thus, this ABC stock is about 60% as volatile as the market.

## Linearized Models - Exponential and Logarithmic Transformations

Some non-linear relationships can be linearized with appropriate transformation

### Exponential Regression

Suppose the relationship between two variables is best described by an exponential function of the form

$$y = a \cdot e^{bx}$$

where $a$ and $b$ are parameters to be found.

Apply $\ln()$ (natural log) to both sides:

$$\ln(y) = \ln(a \cdot e^{bx}) = \ln(a) + \ln(e^{bx}) = \ln(a) + bx$$

Therefore $\ln(y)$ is linear in $x$.

As an example, consider Moore's law stating that the number of transistors per chip would double every 18 months, which results in an exponential model.
The code below uses real data from 1975 to 1995 and compares linear and exponential fits.

The linear fit is clearly very poor as shown in the figure above; still, it is statistically significant with the slope's p-value below 0.05. Therefore, a scatterplot must always be examined.

Now, consider the exponential fit with $\ln(y)$.

The exponential fit in $\ln(y)$ vs $x$ coordinates is very good as shown in the figure above.

Once the linear model is fit, we can express the model in original coordinates and obtain an exponential fit graph in regular $x$-$y$ coordinates as shown in the figure below:

The doubling time $h$ can be found by equating

$$e^{bh} = 2 \implies bh = \ln(2) \implies h = \frac{\ln(2)}{b}$$

Therefore, the doubling time is given by:

It is only a bit higher than Moore's original prediction of 18 months.

### Logarithmic Regression

Another common non-linear model that linearizes well is a power model:

$$y = a \cdot x^b$$

Apply natural log to both sides of this equation and obtain:

$$\ln(y) = \ln(a \cdot x^b) = \ln(a) + \ln(x^b) = \ln(a) + b \ln(x)$$

Therefore $\ln(y)$ is a linear function of $\ln(x)$.

For example, the table below shows the mean distance from the Sun ($x$) in astronomical units (Earth distance is 1) and period ($y$) in Earth years. The code below fits a power model to the data and compares it to the linear model.

The linear fit is very poor again (see figure above) but still statistically significant with a p-value well below 0.05. As always, a scatterplot must be investigated.

The power function fit is very good as shown in the figure above. It gives

$$period (y) = (distance (x))^{1.5}$$

which is exactly Kepler's law.

### S-Shaped (Logistic) Curve

Moore's law in previous sections corresponds to the unabated exponential growth of the number of transistors in computer chips. However, there is a limit to how many transistors can be physically fit in a chip. In a more realistic growth model, the initial exponential growth tapers off. For example, the population of bacteria, the spread of viruses or rumors, the proliferation of global markets, and acceptance of new reform all follow this paradigm. The logistic equation (S-shaped curve) describes such data well:

$$y = \frac{L}{1 + e^{a+bx}}$$

As $x$ increases, $y$ is initially increasing exponentially and concave up, but later tapers off to a concave down curve converging to $L$ that is called the **population carrying capacity** (estimated graphically).
To linearize this equation:

$$\frac{1}{y} = \frac{1 + e^{a+bx}}{L} \implies \frac{L}{y} - 1 = e^{a+bx} \implies \frac{L-y}{y} = e^{a+bx} \implies$$

$$\ln\left(\frac{L-y}{y}\right) = a + bx$$

Therefore $\ln\left(\frac{L-y}{y}\right)$ is linear with $x$.

Consider the growth of bacteria in a limited environment (like a petri dish). The initial exponential growth cannot be sustained. Limited resources and the buildup of toxins decrease the rate of growth. The curve begins to concave up, inflects to concave down, and approaches an asymptotic limit, which looks like an **S-shape**. In these data, the population is converging to about 3000; therefore, the carrying capacity is taken to be $L = 3000$. In the code, we simulated the data adding normal variation to the logistic model.

As before, the linear fit is very poor but statistically significant with a p-value well below 0.05; only the scatterplot above clarifies it. The logistic function fit is very good as shown below.

The Figures above show very good fit, both in the transformed (linearized) and the original coordinates.

## Categorical Predictors

So far, both explanatory ($x$) and response variables ($y$) were numerical. However, the linear regression method can be applied to categorical explanatory variables, but the response variable should still be numerical. The categorical response variable requires a special approach of logistic regression considered in the 2nd part of the book.

**Example**

Investigate linear regression model of cesd by sex in the `HELPrct` data file.

The figures above show a boxplot rather than a scatterplot because the data is categorical. There are only two levels of the predictor variable `sex`, so the residual graph is quite different, but at least the variability is about the same for both groups. The QQ-plot still does not violate normality. It is not a time series, and the Residuals vs. Leverage plot also shows no extreme behavior. Thus, the model can be applied to this data.
Note that the correlation between numerical variable cesd and categorical variable sex cannot be considered. 

The **binary** *yes/no categorical variable* in regression is coded as **1/0 (indicator variable)**. For our data, the base = 0 level is female (F comes before M alphabetically), but this base level can be reset if needed.
Our model $\hat{y} = b_0 + b_1 \cdot x = 36.89 - 5.29 \cdot x$ can now be understood as follows:
For females, $x = sex = 0$; therefore, the **average** predicted cesd score is $\hat{y} = b_0 = 36.89$.
For males, $x = sex = 1$; therefore, the **average** predicted cesd score for males is $\hat{y} = b_0 + b_1 \cdot 1 = b_0 + b_1 = 36.89 - 5.29 = 31.6$.
Thus, for a binary (yes/no) categorical variable, the slope $b_1$ is the average change in the response variable between the two categories. The code below confirms this by computing the means by `sex` (male/female).

The hypothesis test for the slope:

$H_0: \beta_1 = 0$ — zero slope

$H_1: \beta_1 \neq 0$ — non-zero slope

The t-test $t = -3.88$ and *p-value* $= 0.00012 < 0.05$ imply that $H_0$ is rejected and there is a significant slope.

The 95% confidence interval for the slope is $(-7.97, -2.61)$, which does not contain 0 — yet another indication of the non-zero slope.

Let's run the t-test for the comparison of the means with equal variance assumption.

The t-test $t = -3.88$ and *p-value* $= 0.00012 < 0.05$ are the same. The difference in sample means $\bar{x}_{male} - \bar{x}_{female} = 31.6 - 36.89 = -5.29$ is the slope of the regression line $b_1$. Therefore, linear regression for binary categorical variables is equivalent to a t-test with the assumption of equal variance (regression assumes constant variance).

Having confirmed in several ways that the linear model $\hat{y} = 36.89 - 5.29 \cdot x$ is a good fit for the data, use it for prediction. There are only two values of the explanatory variable to predict — female/male; so it just gives average values of the `cesd` score by gender (`sex`).

The typical residual error using this linear model is given by Residual standard error $s_e$ computed below.

Next, consider a categorical variable with more than two levels.

**Example**

Consider the linear regression model for cesd by substance in the `HELPrct` data file.

The figures above show boxplots of the three substances and the diagnostic plots of the model. The substance variable has three levels: alcohol, cocaine, and heroin, so two indicator variables are needed. Alcohol is the base level alphabetically. The cocaine and heroin differences are given by two indicator variables:

$$\hat{cesd} = b_0 + b_{1c} \cdot substance_{cocaine} + b_{1h} \cdot substance_{heroin}$$

$$\hat{cesd} = 34.37 - 4.95 \cdot substance_{cocaine} + 0.50 \cdot substance_{heroin}$$

When a categorical variable has $k$ levels, $k-1$ indicator variables are needed. There are $k-1$ slopes relative to the reference level.

For example, if we consider an alcoholic, then the indicators for cocaine and heroin are both zero: $substance_{cocaine} = substance_{heroin} = 0$, so 

$$cesd = b_0 + b_{1c} \cdot 0 + b_{1h} \cdot 0 = b_0 = 34.37$$

which is the **average** `cesd` depression score for alcoholics.

For cocaine abusers: 

$$\hat{cesd} = b_0 + b_{1c} \cdot 1 + b_{1h} \cdot 0 = b_0 + b_{1c} = 34.37 - 4.95 = 29.42$$

For heroin abusers: 

$$\hat{cesd} = b_0 + b_{1c} \cdot 0 + b_{1h} \cdot 1 = b_0 + b_{1h} = 34.37 + 0.50 = 34.87$$

The depression score means grouped by substance produce exactly the same results.

Therefore, the slopes $b_{1c} = -4.95$ and $b_{1h} = 0.5$ give relative mean differences compared to the reference level of alcohol. The same problem is more naturally considered using the Analysis of Variance (ANOVA) in a separate chapter.

In addition, the linear model output has separate hypothesis tests for each slope.

For cocaine vs. alcohol:

$H_0: \beta_{1c} = 0$ — zero slope for cocaine

$H_1: \beta_{1c} \neq 0$ — non-zero slope

The t-test $t = -3.64$ and p-value $= 0.0003 < 0.05$, so $H_0$ is rejected and there is a significant slope (difference) in average `cesd` scores for cocaine vs. alcohol.

For heroin vs. alcohol:

$H_0: \beta_{1h} = 0$ — zero slope for heroin

$H_1: \beta_{1h} \neq 0$ — non-zero slope

The t-test $t = 0.35$ and p-value $= 0.73 > 0.05$, $H_0$ is not rejected and there is not a significant slope (difference) in average `cesd` scores for heroin vs. alcohol.